In [11]:
import pandas as pd
import re


# Hàm lấy phần sau dấu ":"
def extract_after_colon(line):
    # Tìm dấu ":" và lấy phần sau đó
    match = re.search(r":\s*(.+)", line)
    if match:
        return match.group(1).strip()  # Trả về phần sau dấu ":" (loại bỏ khoảng trắng thừa)
    return line  # Nếu không tìm thấy dấu ":", trả về toàn bộ dòng


def process_csv(input_file, output_file):
    # Đọc file CSV vào DataFrame
    df = pd.read_csv(input_file)

    # Định nghĩa các từ khóa cho từng loại cột
    keywords = {
        "Conference dates": ["conference date", "conference dates"],
        "Location": ["location"],
        "Notification date": ["notification", "review", "released", "acceptance"],
        "Camera-ready date": ["camera", "ready"],
        "Registration date": ["registration", "early", "bird"],
        "Submission date": ["paper", "abstract", "manuscript", "submission", "due", "final", "revision"]
    }

    # Hàm kiểm tra và phân loại từng dòng
    def classify_line(line):
        # Kiểm tra từ khóa
        for col, kw_list in keywords.items():
            for kw in kw_list:
                if re.search(rf"{kw}", line.lower()):
                    print(f"DEBUG: Line '{line}' classified as '{col}'")  # In ra dòng và phân loại
                    return col
        print(f"DEBUG: Line '{line}' classified as 'Others'")  # In ra dòng và phân loại vào 'Others'
        return "Others"  # Nếu không khớp từ khóa nào

    # Tạo danh sách các cột mới
    new_columns = {key: [] for key in keywords.keys()}
    new_columns["Others"] = []

    # Duyệt qua từng phần tử của cột output
    for output in df["output"]:
        # Khởi tạo các danh sách tạm thời để lưu dòng phân loại
        temp_columns = {key: [] for key in new_columns.keys()}
        
        # Bỏ qua dòng đầu tiên bằng cách sử dụng enumerate
        for idx, line in enumerate(output.split("\n")):
            if idx == 0:  # Bỏ qua dòng đầu tiên
                continue
            
            line = line.strip()  # Loại bỏ khoảng trắng thừa
            if line:
                category = classify_line(line)  # Phân loại dòng
                
                # Lấy phần sau dấu ":" chỉ với cột "Conference dates" và "Location"
                if category in ["Conference dates", "Location"]:
                    line = extract_after_colon(line)

                temp_columns[category].append(line)  # Thêm dòng vào cột tương ứng

        # Nối các dòng phân loại và lưu vào các cột mới
        for key in new_columns.keys():
            new_columns[key].append("\n".join(temp_columns[key]))

    # Thêm các cột mới vào DataFrame
    for col, values in new_columns.items():
        df[col] = values

    # Xác định thứ tự cột khi lưu
    column_order = [
        "Conference dates",
        "Location",
        "Submission date",
        "Notification date",
        "Camera-ready date",
        "Registration date",
        "Others"
    ]
    # Đảm bảo giữ nguyên các cột ban đầu
    column_order = ["input", "output"] + column_order

    # Lưu DataFrame đã xử lý ra file mới
    df.to_csv(output_file, columns=column_order, index=False)
    print(f"DEBUG: Data saved to {output_file}")



# Gọi hàm với file đầu vào và đầu ra
input_file = "D:/2023-playwright/geminiapi.csv"
output_file = "./evaluate_core2023.csv"
process_csv(input_file, output_file)


DEBUG: Line 'Conference dates: October 21-23, 2023' classified as 'Conference dates'
DEBUG: Line 'Location: null' classified as 'Location'
DEBUG: Line 'Submission Deadline (last CFP): August 21, 2023' classified as 'Submission date'
DEBUG: Line 'Notification to Authors (last CFP): September 18, 2023' classified as 'Notification date'
DEBUG: Line 'Final Camera-Ready Submission (1st CFP): June 30, 2023' classified as 'Camera-ready date'
DEBUG: Line 'Early Registration (1st CFP): until June 30, 2023' classified as 'Registration date'
DEBUG: Line 'Late Registration (1st CFP): after June 30, 2023' classified as 'Registration date'
DEBUG: Line 'Conference dates: December 11-15, 2023' classified as 'Conference dates'
DEBUG: Line 'Location: The University of Western Australia, Australia' classified as 'Location'
DEBUG: Line 'Deadline for submission of abstracts: November 27, 2023' classified as 'Submission date'
DEBUG: Line 'Early bird registration deadline: November 6, 2023' classified as 'Re